# 构造niid和iid的jsonl，看看其对应的广度和质量

In [1]:
from FlagEmbedding import FlagModel
import numpy as np
from sklearn.manifold import TSNE
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from pprint import pprint as pp
import time
import umap
import os
import random
import time
from contextlib import contextmanager
from datasets import load_dataset, load_from_disk
from datasets import load_dataset, concatenate_datasets, load_from_disk
import pandas as pd
import datasets
from datasets import Dataset
from pprint import pprint as pp
from datasets import Dataset
from sklearn.cluster import KMeans
from tqdm import tqdm
import torch
import heapq

In [2]:
code_data = load_dataset("sahil2801/CodeAlpaca-20k")["train"]
fin_data = load_dataset("FinGPT/fingpt-sentiment-train")["train"]
med_data = load_dataset("medalpaca/medical_meadow_medical_flashcards")["train"]
general_data = load_dataset("tatsu-lab/alpaca")["train"]
math_data = load_dataset("TIGER-Lab/MathInstruct")["train"]
def alpaca_format(example):
    if example['input'] == "":
        example["instruction"] = example["instruction"]
    else:
        example["instruction"] = example["instruction"] + " " + example['input']
    example["response"] = example['output']
    return example
def process_sft_dataset(dataset_name, dataset, dataset_sample=None)->datasets.Dataset:
    if dataset_name in ["lucasmccabe-lmi/CodeAlpaca-20k", "yahma/alpaca-cleaned", "FinGPT/fingpt-sentiment-train"]:
        dataset = dataset.map(alpaca_format, remove_columns=['input', 'output'], desc=f"Preprocessing {dataset_name} for unified format.")
    elif dataset_name in ["WizardLM/WizardLM_evol_instruct_70k"]:
        dataset = dataset.rename_column("output", "response")
    elif dataset_name in ["tatsu-lab/alpaca", "vicgalle/alpaca-gpt4", "gbharti/finance-alpaca"]:
        dataset = dataset.map(alpaca_format, remove_columns=['input', 'output', 'text'], desc=f"Preprocessing {dataset_name} for unified format.")
    elif dataset_name in ["TIGER-Lab/MathInstruct"]:
        df = pd.DataFrame(dataset)
        df = df.drop_duplicates(subset=['instruction'])
        dataset = datasets.Dataset.from_pandas(df)
        dataset = dataset.rename_column("output", "response")
        dataset = dataset.remove_columns(['source'])
    elif dataset_name in ["lighteval/MATH"]:
        dataset = dataset.rename_column("solution", "response")
        dataset = dataset.rename_column("problem", "instruction")
        dataset = dataset.remove_columns(['level', 'type'])
    elif dataset_name in ['gsm8k']:
        dataset = dataset.rename_column("question", "instruction")
        dataset = dataset.rename_column("answer", "response")
    elif dataset_name in ['medalpaca/medical_meadow_medical_flashcards']:       # TODO: 'lavita/ChatDoctor-HealthCareMagic-100k'. not sure whether to discard the instruction.
        dataset = dataset.remove_columns(['instruction'])
        dataset = dataset.rename_column("input", "instruction")
        dataset = dataset.rename_column("output", "response")
    elif "math" in dataset_name:
        dataset = dataset.remove_columns(['source'])
        dataset = dataset.rename_column("output", "response")
    else:
        raise NotImplementedError(f"Dataset {dataset_name} is not supported.")
    dataset = dataset.shuffle(seed=2023)
    if dataset_sample:
        num_sample = min(len(dataset), dataset_sample)
        dataset = dataset.select(range(num_sample))
    print(f">> ===== After processing, Dataset {dataset_name} has {len(dataset)} examples. =====")
    return dataset
processed_data = []
for name, dataset in zip(["lucasmccabe-lmi/CodeAlpaca-20k","FinGPT/fingpt-sentiment-train","medalpaca/medical_meadow_medical_flashcards","tatsu-lab/alpaca","TIGER-Lab/MathInstruct"],[code_data,fin_data,med_data,general_data,math_data]):
# for name, dataset in zip(["lucasmccabe-lmi/CodeAlpaca-20k","FinGPT/fingpt-sentiment-train","medalpaca/medical_meadow_medical_flashcards", "TIGER-Lab/MathInstruct"],[code_data,fin_data,med_data,math_data]):
    tmp:datasets.Dataset = process_sft_dataset(name,dataset)
    print(tmp.column_names)
    processed_data.append(tmp)
    
data_concated = concatenate_datasets(processed_data)

>> ===== After processing, Dataset lucasmccabe-lmi/CodeAlpaca-20k has 20022 examples. =====
['instruction', 'response']
>> ===== After processing, Dataset FinGPT/fingpt-sentiment-train has 76772 examples. =====
['instruction', 'response']
>> ===== After processing, Dataset medalpaca/medical_meadow_medical_flashcards has 33955 examples. =====
['instruction', 'response']
>> ===== After processing, Dataset tatsu-lab/alpaca has 52002 examples. =====
['instruction', 'response']
>> ===== After processing, Dataset TIGER-Lab/MathInstruct has 224567 examples. =====
['response', 'instruction', '__index_level_0__']


In [3]:
model = FlagModel('BAAI/bge-large-en-v1.5', 
                  query_instruction_for_retrieval="",
                  use_fp16=True)

----------using 8*GPUs----------


In [4]:
def matmul_block(A,B,block_size=50000):
    C = torch.zeros(A.size(0), B.size(1))
    # 进行分块矩阵乘法
    for i in range(0, A.size(0), block_size):
        for j in range(0, B.size(1), block_size):
            for k in range(0, A.size(1), block_size):
                # 计算分块索引
                i_end = min(i + block_size, A.size(0))
                j_end = min(j + block_size, B.size(1))
                k_end = min(k + block_size, A.size(1))
                # 执行子块乘法并累加到结果矩阵中
                C[i:i_end, j:j_end] += torch.mm(A[i:i_end, k:k_end].cuda(), B[k:k_end, j:j_end].cuda()).cpu()
    return C


In [5]:
import numpy as np
import torch
from torch.nn.functional import cosine_similarity as cosine_similarity

def coverage(A, V):
    # 将输入转换为张量
    A_tensor = torch.tensor(A, dtype=torch.float32)
    V_tensor = torch.tensor(V, dtype=torch.float32)
    # 计算集合A的覆盖广度
    similarities = matmul_block(V_tensor,A_tensor.T)
    # 计算每个v的最大相似度
    max_similarities = torch.max(similarities, dim=1).values
    # 计算总相似度
    total_similarity = torch.sum(max_similarities).item()
    return total_similarity

In [6]:
data_concated_embeddings = model.encode(data_concated["instruction"])

Inference Embeddings: 100%|██████████| 199/199 [02:20<00:00,  1.42it/s]


## niid的广度

In [7]:
root = "/mnt/bn/data-tns-live-llm/leon/datasets/fed_data/"
datas = []
for i in range(10):
    datas.append(load_from_disk(f"{root}/niid_pos_public_{i}.parquet"))
datas = concatenate_datasets(datas)
print(len(datas))

51000


In [8]:
datas_embeddings = model.encode(datas["instruction"])

Inference Embeddings: 100%|██████████| 25/25 [00:15<00:00,  1.64it/s]


In [43]:
print(coverage(datas_embeddings,data_concated_embeddings))

407318
51000
torch.Size([407318, 51000])
torch.Size([407318, 51000])
torch.Size([407318])
338382.15625


## iid 的广度

In [10]:
root = "/mnt/bn/data-tns-live-llm/leon/datasets/fed_data/"
datas = []
for i in range(10):
    datas.append(load_from_disk(f"{root}/iid_pos_public_{i}.parquet"))
datas = concatenate_datasets(datas)
print(len(datas))
datas_embeddings = model.encode(datas["instruction"])

51000


Inference Embeddings: 100%|██████████| 25/25 [00:19<00:00,  1.26it/s]


In [ ]:
print(coverage(datas_embeddings,data_concated_embeddings))

In [12]:
import numpy as np
import umap
from scipy.spatial import ConvexHull

def umap_embeddings_volume(embeddings, n_neighbors=10, n_components=3):
    # 初始化 UMAP 降维模型
    reducer = umap.UMAP(n_neighbors=n_neighbors, n_components=n_components)
    # 进行 UMAP 降维
    low_dim_embeddings = reducer.fit_transform(embeddings)
    # 将降维后的数据转换为 NumPy 数组
    points = np.array(low_dim_embeddings)
    # 计算凸包
    hull = ConvexHull(points)
    return hull.volume

In [13]:
# 调用函数计算 UMAP 降维后凸包的体积
volume = umap_embeddings_volume(datas_embeddings)
print("The volume of the UMAP-reduced embeddings is:", volume)

The volume of the UMAP-reduced embeddings is: 6339.46117174317
